### 配置文件验证

In [1]:
from omegaconf import OmegaConf
hparams = OmegaConf.load('config.yaml') # 读取配置文件

In [4]:
from modelModule.model4 import VAE4
import torch

In [5]:
model = VAE4(dim=57, nhead=3)
a = torch.randn((10,57))
b = torch.randn((10,57))
x, y, z = model(a,b)

In [6]:
x

tensor([[ 0.4962,  0.9074, -0.6625,  0.2056,  0.8354,  0.7072, -0.8985,  0.3271,
          0.3512,  0.6712,  0.7555, -0.6314,  0.1071, -0.8705, -0.6888,  0.9757,
          0.1672, -1.5787,  0.3462,  0.0699,  0.2273,  0.1713,  0.7905, -0.3541,
         -0.9389, -0.4416,  0.7132,  1.4633,  0.2812,  0.5111,  0.4218, -0.6275,
         -0.4900,  0.2257, -0.3728, -0.0539, -0.3407,  0.5292,  0.3222, -1.5075,
          0.7729, -0.8541, -0.0021,  0.2028, -1.0716, -0.1460,  0.1415, -0.0072,
          1.1094, -0.2342,  0.3430,  1.8601,  1.5920,  0.0365,  0.2995,  0.9036,
          0.8384],
        [ 0.2733,  1.0036, -0.7167,  0.3102,  0.7463,  0.8064, -0.8257,  0.3728,
          0.2798,  0.7997,  0.7051, -0.6529,  0.0225, -0.9354, -0.6999,  0.9143,
          0.3978, -1.6326,  0.1396,  0.2251,  0.2236,  0.0848,  0.8768, -0.5991,
         -0.8477, -0.5038,  0.7001,  1.5412,  0.4138,  0.6183,  0.4880, -0.5676,
         -0.3281,  0.1450, -0.3144,  0.0980, -0.2153,  0.4346,  0.2722, -1.5265,
         

### dataloader验证

In [ ]:
import torch
from dataModule.dataset1 import FlatDataset
from torch.utils.data import DataLoader

In [ ]:
dataset = FlatDataset('Spam_train.csv')
print(dataset.Max_Val.shape)
print(dataset.Min_Val)

In [ ]:
dataset = FlatDataset('Letter_train.csv')
train_dataloader = DataLoader(dataset, 
                             batch_size=10, 
                             collate_fn=dataset.collater, 
                             shuffle=True,
                             num_workers=2)
batch = None
for i in train_dataloader:
    batch = i
    break

In [ ]:
test_data = imputed_data * global_max + global_min # 广播效应

In [ ]:
a = torch.randn(2,3,4)
b = torch.nn.AdaptiveMaxPool1d(output_size=1)(a).squeeze(-1)

In [ ]:
b.shape

### 模型精度测定

In [5]:
from dataModule.dataset1 import FlatDataset
# from modelModule.model1 import VAE
from modelModule.model2 import VAE2
from torch.utils.data import DataLoader
from utils import restore_data, result_show, get_missing
import torch
import numpy as np

In [6]:
## 模型参数载入
model = VAE2(dim=57, nhead=3)
checkpoint_path = r'模型参数保存/Spam_model2_norm_way_mean_norm_/version_1/checkpoints/epoch=126-step=1905.ckpt'    
data_norm = 'mean_norm'  # 数据集采用正则化方式minmax_norm,mean_norm
train_csv = r'Spam_train.csv'
test_csv = r'Spam_test.csv'

checkpoint = torch.load(checkpoint_path)
model.load_state_dict({k.replace('model.',''):v for k,v in checkpoint['state_dict'].items()}) 
model.eval()
## 载入训练集的均值(最小)方差(最大)
train_dataset = FlatDataset(train_csv, data_norm=data_norm)
model.get_global_min_max(global_max=train_dataset.Max_Val, global_min=train_dataset.Min_Val)
print('载入成功')

载入成功


In [7]:
test_dataset = FlatDataset(test_csv, data_norm=data_norm)
data_size = 10
src_data = np.stack([test_dataset[index]['src_data'] for index in range(data_size)], axis=0)
normal_data = np.stack([test_dataset[index]['global_normal_data'] for index in range(data_size)], axis=0)
_, miss_matrix = get_missing(src_data, 0.3) # 得到缺失矩阵
input = src_data.copy()
input[miss_matrix == 0] = np.nan

In [9]:
imputed_data, output = model.inference(input, miss_matrix) # inference输出的结果已经重构后的结果
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()

torch.Size([10, 57])
torch.Size([10, 57])


In [12]:
result_show(src_data, imputed_data, miss_matrix)

原始数据 [5.000e-01 7.000e-01 0.000e+00 1.610e+00 1.110e+00 0.000e+00 6.000e-01
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 1.000e-01 1.000e-01 0.000e+00 6.690e+02 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 5.520e-01 1.000e+00 1.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.666e+00
 4.000e+00 8.200e-01 3.200e-01 1.600e-01 6.500e-01 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 2.900e-01 2.900e-02 1.580e+02
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 2.200e-02 2.200e-02 2.200e-02 3.482e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 

array([7.13741863e-01, 5.30362276e-02, 7.81178994e+06, 9.16165401e-01,
       9.13319763e-01, 4.84912365e+07, 8.81842240e-01, 1.78035260e+07,
       1.47751521e+06, 7.25282804e+06, 9.48811227e+06, 1.90410121e+07,
       3.73398258e+06, 1.89243126e+07, 4.19950704e+07, 4.30696270e+00,
       1.85022394e-01, 2.39612393e+06, 9.81040936e-01, 9.70458798e+07,
       1.37408588e+07, 5.43569469e+07, 6.64607139e+07, 5.71509828e+06,
       7.80015570e+06, 1.34398547e+07, 4.83860872e+07, 1.79001769e+07,
       1.93128899e+06, 7.26734636e+06, 2.45512775e+06, 1.06902382e+00,
       2.65633650e+00, 1.04265403e+01, 9.61281124e+07, 1.37252434e+07,
       5.39191696e+07, 7.63046948e+06, 3.29236043e+07, 9.67499137e+06,
       2.06144942e+08, 6.35894260e+07, 7.79717265e+07, 1.75261367e+07,
       3.69528417e+06, 7.94503309e+06, 1.47390844e+05, 1.02908680e+07,
       1.18115892e+00, 1.92212273e+00, 1.80810086e-01, 5.69541519e-01,
       1.38564365e+00, 1.62424385e-01, 5.67100662e+06, 1.34949489e+07,
      

In [10]:
imputed_data

array([[ 2.14650995e-01,  9.66677355e-01,  1.59321301e-02,
         3.07760979e-01,  2.61114240e-01,  1.37836326e-01,
         1.43129061e-01, -6.14348519e-02, -9.17590184e-02,
         5.44047840e-01,  9.21930866e-03,  6.62874640e-01,
        -5.63370100e-02, -7.81178994e-02,  1.34973695e-01,
         6.10303376e-01,  3.26908852e-01,  9.62150538e-02,
         2.05998535e+00, -6.67448819e-02,  6.33488838e-01,
         4.84912365e-01,  7.08946473e-02,  1.70145506e-01,
         7.71645127e-01,  1.70566600e-01,  8.15368326e-01,
        -1.78035260e-01,  2.42935520e-01, -1.47751521e-02,
        -1.10080786e-01,  1.26534516e-01,  3.97732235e-01,
        -7.25282804e-02,  9.48811227e-02,  2.01409670e-01,
         1.90410121e-01, -4.42566302e-02,  7.68581503e-02,
         3.73398258e-02,  1.89243126e-01, -3.19244922e-01,
        -3.33930650e-02,  4.19950704e-01,  5.30696313e-01,
         8.14977587e-02,  2.39612393e-02, -1.38583269e-03,
         1.17227848e-01, -3.66001428e-02,  6.56482465e-0

In [ ]:
imputed_data.shape

In [ ]:
# test_dataset = FlatDataset('Letter_test.csv', data_norm='mean_norm') # minmax_norm
# index = 102
# src_data, M_matrix, miss_data = test_dataset[index]['src_data'].reshape(1,-1), test_dataset[index]['global_miss_matrix'].reshape(1,-1), test_dataset[index]['global_miss_data'].reshape(1,-1)
# src_data = test_dataset[index]['src_data'].reshape(1,-1)
# normal_data = test_dataset[index]['global_normal_data'].reshape(1,-1)
# miss_data, M_matrix = get_missing(normal_data, 0.3) # 产生掩码数据，并采用随机数填补

# imputed_data, _, _ = model(torch.from_numpy(miss_data).float(), torch.from_numpy(M_matrix).float()) 
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy() # 得到最终插补结果,数据由[0,1]范围内
# res_imputed_data = np.around(restore_data(imputed_data, test_dataset.Max_Val, test_dataset.Min_Val))
# result_show(src_data, res_imputed_data, M_matrix)

In [ ]:
# batch测试
# src_data, miss_data, M_matrix = batch['src_data'], batch['miss_data'], batch['miss_matrix']
# normal_data = batch['normal_data']
# imputed_data, _, _ = model(miss_data, M_matrix)
# imputed_data = M_matrix * miss_data + (1-M_matrix) * imputed_data.data.numpy()
# res_imputed_data = np.around(restore_data(imputed_data, dataset.Max_Val, dataset.Min_Val))